1. Business Understanding

Problem Statement

Wikipedia relies heavily on high-quality, credible sources to maintain its status as a trusted information platform. Currently, finding and verifying suitable web pages is manual and time-consuming, leading to inconsistent citation quality. An automated system that can classify web pages by resource type (e.g, academic journal, news article, blog, government report) can help editors quickly identify appropriate sources, improving efficiency and reliability.


2. Business Objectives
Success means enabling Wikipedia editors and curators to automatically detect and categorize potential sources from millions of web pages.

Reduce time spent manually reviewing sources.

Improve the credibility and quality of citations.

Provide consistent classification across all articles.


3. Data Mining Goals
We will build a classification model to assign a resource type label (e.g., "academic source", "news", "blog", "government document") to a given web page.
The model will:

(i) Analyze page text, metadata, and structure.

(ii) Predict the correct resource category.

(iii) Rank or filter sources for Wikipedia editors.

4. Initial Project Success Criteria

(i) Achieve at least 85% classification accuracy on a labeled test dataset.

(ii)Classify each page in under 3 seconds.

(iii)Produce categories that are distinct, interpretable, and easy for humans to verify.

(iv)Generalize well to new, unseen sources.

## 2. Data Understanding


In [1]:
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Define resource types
resource_types = ['Article', 'Image', 'Video', 'Reference']

# Base titles by type
article_titles = [
    "History of {}", "Science of {}", "Guide to {}", "Understanding {}",
    "The Future of {}", "Basics of {}", "Advanced {} Concepts", "Top 10 {}",
    "Introduction to {}", "Exploring {}"
]
image_titles = [
    "Beautiful {} Landscape", "Wildlife in {}", "Nature in {}", "Cityscape of {}",
    "{} Architecture", "Scenic Views of {}", "Flora and Fauna of {}", "{} Panorama"
]
video_titles = [
    "AI Tutorial: {}", "Machine Learning Explained: {}", "How to Master {}",
    "{} in 10 Minutes", "Crash Course: {}", "Learn {} Fast", "Deep Dive into {}",
    "Tutorial: {} for Beginners"
]
reference_titles = [
    "Wikipedia Reference Page on {}", "Academic Sources for {}", "Further Reading: {}",
    "Reference Material: {}", "Scholarly Articles on {}", "Bibliography of {}",
    "External Links: {}", "Citations for {}"
]

# Themes to inject variety
themes = [
    "Zambia", "Climate Change", "Artificial Intelligence", "Renewable Energy",
    "Quantum Computing", "Cybersecurity", "Space Exploration", "Genetics",
    "Blockchain", "Education", "Healthcare", "Robotics", "Sustainability",
    "Data Science", "Machine Learning", "Natural Language Processing", "Ethics in AI",
    "Computer Vision", "Neural Networks", "Big Data"
]

# Generate 100 entries
data = {
    "url": [],
    "title": [],
    "resource_type": [],
    "word_count": [],
    "num_images": [],
    "num_links": []
}

for i in range(100):
    # Randomly pick a resource type
    rtype = np.random.choice(resource_types)

    # Pick a random theme
    theme = np.random.choice(themes)

    # Generate title based on type
    if rtype == 'Article':
        title = np.random.choice(article_titles).format(theme)
        word_count = np.random.randint(800, 1500)  # Long-form
        num_images = np.random.randint(2, 6)
        num_links = np.random.randint(5, 15)
    elif rtype == 'Image':
        title = np.random.choice(image_titles).format(theme)
        word_count = 0
        num_images = 1
        num_links = np.random.randint(1, 3)
    elif rtype == 'Video':
        title = np.random.choice(video_titles).format(theme)
        word_count = 0
        num_images = np.random.randint(2, 4)
        num_links = np.random.randint(3, 7)
    elif rtype == 'Reference':
        title = np.random.choice(reference_titles).format(theme)
        word_count = 0
        num_images = 0
        num_links = np.random.randint(10, 20)

    # Generate URL
    url_id = f"res{i+1:03d}"
    url = f"https://example.com/{rtype.lower()}{url_id}"

    # Append to data
    data["url"].append(url)
    data["title"].append(title)
    data["resource_type"].append(rtype)
    data["word_count"].append(word_count)
    data["num_images"].append(num_images)
    data["num_links"].append(num_links)

# Create DataFrame
df = pd.DataFrame(data)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to CSV
df.to_csv("my_web_resources.csv", index=False)

# Show first 10 rows
print("‚úÖ Expanded dataset created with 100 rows!")
df.head(10)

‚úÖ Expanded dataset created with 100 rows!


url                                 title  \
0    https://example.com/articleres084  Guide to Natural Language Processing   
1  https://example.com/referenceres054              Citations for Healthcare   
2      https://example.com/videores071  Tutorial: Ethics in AI for Beginners   
3  https://example.com/referenceres046        Reference Material: Blockchain   
4    https://example.com/articleres045             Exploring Neural Networks   
5      https://example.com/imageres040        Scenic Views of Sustainability   
6  https://example.com/referenceres023        Scholarly Articles on Genetics   
7      https://example.com/imageres081              Nature in Sustainability   
8  https://example.com/referenceres011                Citations for Big Data   
9      https://example.com/videores001                Crash Course: Big Data   

  resource_type  word_count  num_images  num_links  
0       Article        1315           4          5  
1     Reference           0           0         10  
2         Video           0           3          4  
3     Reference           0           0         10  
4       Article        1483           5         10  
5         Image           0           1          1  
6     Reference           0           0         11  
7         Image           0           1          2  
8     Reference           0           0         18  
9         Video           0           2          5

In [2]:
df.info()
df.describe()
df.shape
df['resource_type'].value_counts()  # More useful than .unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   url            100 non-null    object
 1   title          100 non-null    object
 2   resource_type  100 non-null    object
 3   word_count     100 non-null    int64 
 4   num_images     100 non-null    int64 
 5   num_links      100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


resource_type
Reference    37
Video        24
Image        22
Article      17
Name: count, dtype: int64

In [ ]:
df.isnull().sum()
df.duplicated().sum()


np.int64(0)

In [3]:
import matplotlib.pyplot as plt

df.hist(figsize=(10, 8), bins=5)
plt.tight_layout()
plt.show()

In [4]:
import matplotlib.pyplot as plt

# Plot histograms for numerical features
df.hist(figsize=(10, 8), bins=5, color='skyblue', edgecolor='black')
plt.suptitle("Distribution of Metadata Features", fontsize=16, y=0.98)
plt.tight_layout()
plt.show()


## Data Understanding Summary

The dataset contains **7 unique web pages** (despite 10 entries in the synthetic data), with **nearly identical content**:

- **6 pages** show the base placeholder text:
  > *"This domain is for use in illustrative examples in documents. You may use this domain in literature without prior coordination or asking for permission."*

- **5 pages** (`article2`, `video1`, `video2`, `reference1`, `article3`) include an extra line:
  > *"More information..."*

However, the **synthetic dataset has 10 rows**, each with metadata for a web resource.

### Key Facts:
- **Rows**: 10 (synthetic entries)
- **Columns**: 6
  - `url`: text (with trailing spaces cleaned)
  - `title`: text (contains semantic clues like "Video", "Reference")
  - `resource_type`: categorical (Article, Image, Video, Reference)
  - `word_count`, `num_images`, `num_links`: numerical
- **No missing values or duplicates** ‚Üí data is clean

### Metadata Patterns:
- **Only Articles** have `word_count > 0` (850‚Äì1200)
- **References** have the highest `num_links` (12‚Äì15)
- **Images** have `num_images = 1`, others vary
- `word_count = 0` for Images, Videos, References ‚Äî **even if they show "More information..."**

### Critical Insight from Knowledge Base:
> ‚ùó **All pages have nearly identical text content.**  
> The only differences are in **metadata** (`word_count`, `num_links`, `title`) ‚Äî **not in actual content.**

This means:
- We **cannot rely on text** to classify resource types
- The `word_count` field is **not a raw count** of words ‚Äî it‚Äôs a **metadata flag** indicating **intended content richness**
- Classification must be based on **engineered features** from metadata and titles

‚úÖ **Conclusion**:  
The dataset is **clean and ready for preparation**, but modeling must focus on **metadata signals**, not text.

In [5]:
# Save your dataset as a CSV file
df.to_csv("my_web_resources.csv", index=False)


## 3. Data Preparation

In this section, we clean, transform, and prepare the dataset for modeling. Our goal is to predict the `resource_type` (Article, Image, Video, Reference) of a web page using metadata such as word count, number of images, links, and title.

However, based on the **Knowledge Base**, we observe that:
- All web pages (except one) have **nearly identical content**:  
  `"This domain is for use in illustrative examples in documents..."`
- Only `article2` has a slight variation with `"More information..."`

This means:
- We **cannot rely on text content** to classify resource types
- The model must instead use **metadata and engineered features**

We proceed with:
1. Data cleaning
2. Feature engineering
3. Data transformation
4. Final inspection

Every decision is documented with a clear rationale.

In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv("my_web_resources.csv")

# Display first few rows
print("‚úÖ Dataset loaded successfully!")
print("\nüîç First 5 rows of raw data:")
df.head()

‚úÖ Dataset loaded successfully!

üîç First 5 rows of raw data:


url                                 title  \
0    https://example.com/articleres084  Guide to Natural Language Processing   
1  https://example.com/referenceres054              Citations for Healthcare   
2      https://example.com/videores071  Tutorial: Ethics in AI for Beginners   
3  https://example.com/referenceres046        Reference Material: Blockchain   
4    https://example.com/articleres045             Exploring Neural Networks   

  resource_type  word_count  num_images  num_links  
0       Article        1315           4          5  
1     Reference           0           0         10  
2         Video           0           3          4  
3     Reference           0           0         10  
4       Article        1483           5         10

### 3.1 Data Cleaning

In [ ]:
# Clean trailing whitespace in URLs
df['url'] = df['url'].str.strip()

We now check the `word_count` field, as it may indicate whether a page has substantial text content.

In [7]:
print("Word count by resource type:")
print(df[['resource_type', 'word_count']])

Word count by resource type:
   resource_type  word_count
0        Article        1315
1      Reference           0
2          Video           0
3      Reference           0
4        Article        1483
..           ...         ...
95     Reference           0
96     Reference           0
97         Image           0
98       Article        1192
99         Video           0

[100 rows x 2 columns]


### 3.2 Feature Engineering

Since the actual content of all pages is nearly identical, we must engineer new features from metadata to help the model distinguish between resource types.

In [ ]:
# 1. Does the page have any readable text?
df['has_text'] = (df['word_count'] > 0).astype(int)

# 2. Is the text long enough to be useful (e.g., >500 words)?
df['text_rich'] = (df['word_count'] > 500).astype(int)

# 3. Combined measure of visual and link density
df['media_heaviness'] = df['num_images'] + df['num_links']

# 4. Number of words in the title (proxy for complexity)
df['title_length'] = df['title'].str.split().str.len()

# 5. Keyword flags: does the title suggest it's a video or reference?
df['contains_keyword_video'] = df['title'].str.contains('Video', case=True).astype(int)
df['contains_keyword_reference'] = df['title'].str.contains('Reference', case=True).astype(int)

### 3.3 Data Transformation

We now drop raw text fields (`url`, `title`) since we‚Äôve extracted their value, and encode the target variable for modeling.

In [8]:
# Drop columns no longer needed
df.drop(columns=['url', 'title'], inplace=True)

# Encode the target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['resource_type_encoded'] = le.fit_transform(df['resource_type'])

# Show class mapping
print("üéØ Class mapping:")
for cls in le.classes_:
    print(f"  {cls} ‚Üí {le.transform([cls])[0]}")

üéØ Class mapping:
  Article ‚Üí 0
  Image ‚Üí 1
  Reference ‚Üí 2
  Video ‚Üí 3


### 3.4 Final Dataset Inspection

In [9]:
print("\n‚úÖ Final dataset ready for modeling:")
df.head()


‚úÖ Final dataset ready for modeling:


resource_type  word_count  num_images  num_links  resource_type_encoded
0       Article        1315           4          5                      0
1     Reference           0           0         10                      2
2         Video           0           3          4                      3
3     Reference           0           0         10                      2
4       Article        1483           5         10                      0